In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od

od.download('https://www.kaggle.com/datasets/ingbiodanielh/vizwiz')

In [3]:
!ls /content/vizwiz/data/Images | wc -l

31173


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [1]:
import clip
import json
import numpy as np
import torch
import torchtext
from PIL import Image
import torch.nn as nn
from sklearn.model_selection import train_test_split

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
img = Image.open('/content/vizwiz/data/Images/VizWiz_train_000000000000.jpg')

In [ ]:
img.show()

In [2]:
data = None

with open('/content/vizwiz/data/Annotations/train.json') as f:
  data = json.load(f)

In [3]:
len(data)

20000

In [4]:
d = {'img': 0, 'answerable': 1, 'question': 2, 'answer_type': 3, 'answer': 4}

In [5]:
df = []

for entry in data:
  df.append([entry['image'], entry['answerable'], entry['question'], entry['answer_type'], entry['answers'][4]['answer']])

df = np.array(df)

In [6]:
df.shape

(20000, 5)

In [7]:
y = np.array([(entry[3], entry[4]) for entry in df])

In [29]:
answers = np.array([entry[4] for entry in df])

In [30]:
answer_types = np.array([entry[3] for entry in df])

In [8]:
y.shape

(20000, 2)

In [9]:
y[10]

array(['other', 'samsung phone'], dtype='<U92')

In [10]:
model, preprocess = clip.load('ViT-B/32', device=device)

In [27]:
X = []

with torch.no_grad():
  for i in range(3000):
    encoded_img = model.encode_image(preprocess(Image.open('/content/vizwiz/data/Images/'+df[i][0])).unsqueeze(0).to(device)).to('cpu')
    encoded_q = model.encode_text(clip.tokenize(df[i][2]).to(device)).to('cpu')

    X.append([encoded_img, encoded_q])

X = np.array(X)

<ipython-input-27-8599fef85aa1>:10: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  X = np.array(X)
<ipython-input-27-8599fef85aa1>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(X)


In [39]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenized_answers = np.array([tokenizer(answer) for answer in answers])
print(tokenized_answers[10])

['samsung', 'phone']


<ipython-input-39-6aad3a95851a>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tokenized_answers = np.array([tokenizer(answer) for answer in answers])


In [40]:
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_answers, min_freq=1)
vocab.insert_token('<unk>', 0)
vocab.insert_token('<eos>', 1)
vocab.set_default_index(vocab['<unk>'])
print(len(vocab))
print(vocab.get_itos()[:10])

6048
['<unk>', '<eos>', 'unanswerable', 'unsuitable', 'no', 'white', 'yes', 'black', 'blue', 'grey']


In [41]:
fin_answers = []

for answer in tokenized_answers:
  if answer:
    answer.append('<eos>')
    tokens = [vocab[token] for token in answer]
    fin_answers.append(tokens)

print(fin_answers[10])

[616, 18, 1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Custom Dataset class
class VQADataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        label = self.labels[index]

        return x, label

In [ ]:
train_set = DataLoader(VQADataset(X_train, y_train), batch_size=32)
test_set = DataLoader(VQADataset(X_test, y_test), batch_size=32)

In [ ]:
class VQA_Network(nn.Module):
    def __init__(self, num_classes, seq_len, hidden_dim):
        super(VQA_Network, self).__init__()

        self.fc = nn.Linear(embedding_dim, hidden_dim)
        self.fc2_answers = nn.Linear(hidden_dim, seq_len)

        self.fc2_aux = nn.Linear(hidden_dim, num_classes)
        self.fc3_aux = nn.Linear(num_classes, seq_len)

        self.norm = nn.LayerNorm(hidden_dim)
        self.dropout = nn.Dropout()

    def forward(self, x):
        # x = x.to(dtype=torch.float32)
        x = self.fc(x)
        x = self.norm(x)
        x = self.dropout(x)

        qtype = self.fc2_aux(x)
        aux = self.fc3_aux(qtype)

        answers = self.fc2_answers(x)

        answers = answers * aux

        # answers = answers.to(dtype=torch.int64)

        return answers, qtype


In [ ]:
num_classes = 5726
seq_len = 128
hidden_dim = 512

# Instantiate the model
model = VQA_Network(num_classes, seq_len, hidden_dim).to(device)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
for epoch in range(200):
    model.train()
    running_loss_a = 0.0
    running_loss_q = 0.0

    for xs, labelset in train_set:
        # Move data to the device
        xs = xs.to(device)
        labelset = labelset.to(device)
        answerset, qtypeset = labelset

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        answers, qtypes = model(xs)
        print(xs.size())
        print(labelset.size())
        # Compute loss
        answer_loss = criterion(answers, answerset)

        # Backward pass
        answer_loss.backward()

        # Compute loss
        qtype_loss = criterion(qtypes, qtypeset)

        # Backward pass
        qtype_loss.backward()

        # Update weights
        optimizer.step()

        # Update running loss
        running_loss_a += answer_loss.item()
        running_loss_q += qtype_loss.item()

    # Calculate average loss for the epoch
    average_loss_a = running_loss_a / len(data_loader)
    average_loss_q = running_loss_q / len(data_loader)

    # Print progress
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss_a: {average_loss_a:.4f}, Loss_q: {average_loss_q:.4f}")

# Training complete
